<a href="https://www.kaggle.com/dinoatkaggle/diabetes-prediction-optimizing-recall-b28b2d?scriptVersionId=86713779" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Dataset Description

The data is taken from "https://www.kaggle.com/mathchi/diabetes-data-set"

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases.All the patients here are female 21 years or older.It contains the following columns:
 
  - Pregnancies: Number of times pregnant
  - Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
  - BloodPressure: Diastolic blood pressure (mm Hg)
  - SkinThickness: Triceps skin fold thickness (mm)
  - Insulin: 2-Hour serum insulin (mu U/ml)
  - BMI: Body mass index (weight in kg/(height in m)^2)
  - DiabetesPedigreeFunction: Diabetes pedigree function
  - Age: Age (years)
  - Outcome: Class variable (0 or 1)

# Objectives

- Clean the data and deal with missing values
- Perform EDA to get a better understanding of underlying trends
- Fit different models and tune their hyperparameter for best performance

# Data Cleaning

In [ ]:
data = pd.read_csv("../input/diabetes-data-set/diabetes.csv")

In [ ]:
data.head()

In [ ]:
print("Data has\033[1m" , data.shape[0] , "\033[0mrows and\033[1m", data.shape[1] , "\033[0mcolumns\n\n")
print("Column Name \t\t Data Type\n")
print(data.dtypes) 

In [ ]:
data.isnull().sum() # no missing values at first glance

In [ ]:
data.duplicated().sum() #no duplicated values

In [ ]:
data.describe()

We note that minimum value for columns such as Glucose,BP,Insulin,BMI are 0 which is not possible , hence these must be missing values which are encoded as 0
We can use imputation techniques to deal with these

In [ ]:
corrupted_data = data.loc[:,'Glucose':'BMI']

In [ ]:
corrupted_data.replace(0,np.nan,inplace=True) # replace 0 with NaN

In [ ]:
data.loc[:,'Glucose':'BMI'] = corrupted_data

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

Lets observe these missing values more closely and find out if their is any relation

In [ ]:
import missingno as msno

In [ ]:
msno.bar(data)

In [ ]:
msno.matrix(data)

In [ ]:
msno.matrix(data.sort_values(by="Insulin"))

Looking at the above matrix we can figure out that glucose and BMI are MCAR(Missing completely at random) and insulin , SkinThickness are MNAR(Missing not at random)

In [ ]:
msno.heatmap(data)

We can see heavy correlation between SkinThickness and Insulin

In [ ]:
data.dropna(subset=['Glucose','BMI'],inplace=True) # drop rows having missing values for Glucose or BMI

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
data_knn = data.copy()

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5)
data_knn.iloc[:,:] = knn_imputer.fit_transform(data_knn)

Using knnimputer we fill the missing values without impacting the variability of the dataset

In [ ]:
data_knn.shape

In [ ]:
data_knn.describe()

# Outlier Removal

In [ ]:
cleaned_data = data_knn.copy()

In [ ]:
sns.boxplot(data=data_knn,x='Pregnancies')

Values above 13 are outliers and removed 

In [ ]:
cleaned_data = cleaned_data[cleaned_data['Pregnancies']<=13]

In [ ]:
sns.boxplot(data=cleaned_data,x='Pregnancies')

Verify the outlier removal

In [ ]:
sns.boxplot(data=data_knn,x='Glucose')

No outliers detected

In [ ]:
sns.boxplot(data=data_knn,x='BloodPressure')

BP below 30 mm Hg can be considered as outliers and hence removed

In [ ]:
cleaned_data = cleaned_data[cleaned_data['BloodPressure']>=30]

In [ ]:
sns.boxplot(data=cleaned_data,x='BloodPressure')

Verify outlier removal effect

In [ ]:
sns.boxplot(data=data_knn,x='Insulin')

Insulin level above 500 could skew our model and hence removed

In [ ]:
cleaned_data = cleaned_data[cleaned_data['Insulin']<=500]

In [ ]:
sns.boxplot(data=data_knn,x='SkinThickness')

SkinThickness of 99 removed as outlier

In [ ]:
cleaned_data = cleaned_data[cleaned_data['SkinThickness']<=80]

In [ ]:
sns.boxplot(data=data_knn,x='BMI')

BMI obove 60 removed 

In [ ]:
cleaned_data = cleaned_data[cleaned_data['BMI']<=60]

In [ ]:
sns.boxplot(data=data_knn,x='Age')

Age of 81 is removed as outlier

In [ ]:
cleaned_data = cleaned_data[cleaned_data['Age']<=70]

In [ ]:
print("We removed",data_knn.shape[0]-cleaned_data.shape[0],"outliers")

# Exploratory Data Analysis

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(cleaned_data.corr(),annot=True,cmap='viridis')

The following conclusions can be drawn from the heatmap :
- There is not much correlation among different predictors
- Age and Pregnancy have a positive corr indicating that adults have more children
- There is high corr between BMI and SkinThickness, generally people with very high BMI are considered obese thus explaining thicker skin
- There is positive correlation among Insulin and Glucose as well which could be explained by the fact that perhaps the type 1 diabetic patients who generally have high Glucose , were given Insulin injections.
- Glucose also has corr with our Outcome , i.e. , diabetic patients have higher level of glucose in their blood

In [ ]:
sns.set_style('white')
sns.histplot(data=cleaned_data,x='Age',y='Pregnancies',color='darkblue')

Females between age of 40 and 50 have more children while those around the age of 21 generally have 0 children

In [ ]:
sns.displot(data=cleaned_data,x='Pregnancies',hue='Outcome',kde=True)

Majority of females taking part have 0 preganancies , also the number of pregnancies doesn't seem to have impact on wether someone is diabetic or not

In [ ]:
sns.displot(data=cleaned_data,x='Glucose',hue='Outcome',kind='kde')

People who don't have diabeties have glucose normally distributed around 100 as mean , while those with diabeties have much higher level of glucose ranging between 100 and 200

In [ ]:
sns.scatterplot(data=cleaned_data,x='Glucose',y='Insulin',hue='Outcome')

Type 1 diabeties is caused when pancreas produces very little to no insulin and thus the level of glucose increases , from the scatterplot we can clearly see that diabetic patients have high glucose and low insulin 

##### Note: Data is collected after 2 hour of giving an insulin injection which explains the high values of insulin.

In [ ]:
sns.displot(data=cleaned_data,x='BloodPressure',hue='Outcome',kind='kde')

BloodPressure seems to be normally distributed for both positive and negative cases, indicating little corr as we have seen earlier. The mean BloodPressure seems to be a bit higher but is still within the normal diastolic range of 60-80 mm Hg

In [ ]:
sns.scatterplot(data=cleaned_data,y='SkinThickness',x='BMI',hue='Outcome')

BMI range of 18.5-24.9 is generally considered as normal while those above 30 are considerd as obsese ans BMI above 55 is considerd as Morbid Obsese
With increase in BMI skinthickness also increases and it seems that obses people are generally more prone to diabeties

In [ ]:
cleaned_data['Outcome'].value_counts(normalize=True)*100

The dataset seems to be imbalanced with 65% of input as non diabetic , we will use SMOTE to deal with this later as we train our model

# Metric
It is important to identify the correct metric for a classification problem with unbalanced dataset as accuracy would fail 
Since we are trying to build a model to corretly identify patients with Diabeted we should prioritize our recall or f1 score.

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [ ]:
X = cleaned_data.drop('Outcome',axis=1)  # predictor variables
y = cleaned_data['Outcome']  # outcome variable

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y,random_state=42) #with stratify=True we get same proportion in our train test split as in our original dataset

In [ ]:
sm = SMOTE(random_state=42) # initalize SMOTE
x_res,y_res=sm.fit_resample(X_train,y_train) # Oversample our data.

In [ ]:
#select best params
from sklearn.model_selection import GridSearchCV
estimator = Pipeline([("scaler", StandardScaler()),
        ("logistic_regression", LogisticRegression(solver='liblinear'))])

params = {'logistic_regression__penalty' : ['l1','l2'],
    'logistic_regression__C': np.linspace(0.001,100)
}
grid = GridSearchCV(estimator, params, cv=5,verbose=True,n_jobs=-1,scoring='recall')

In [ ]:
grid.fit(x_res,y_res) #fit the model

In [ ]:
grid.best_params_ #best params

In [ ]:
y_pred = grid.predict(X_test)  #predict 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
#plot the confusion matrix
_, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='viridis', annot_kws={"size": 40, "weight": "bold"})  
labels = ['True', 'False']
ax.set_xticklabels(labels, fontsize=25);
ax.set_yticklabels(labels, fontsize=25);
ax.set_ylabel('Prediction', fontsize=30);
ax.set_xlabel('Ground Truth', fontsize=30)

In [ ]:
print(classification_report(y_test,y_pred))

A recall of 0.75 is not that bad , lets see if we can improve upon in other models

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
se = StandardScaler()  # we need to scale our data for knn
X_scaled = se.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33,stratify=y,random_state=42)

In [ ]:
sm = SMOTE(random_state=42)
x_res,y_res=sm.fit_resample(X_train,y_train)

In [ ]:
#to optimize the value of k 
from sklearn.metrics import recall_score,f1_score
max_k = 40
recall_scores = list()
f1_scores = list()
for k in range(1, max_k):
    
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
    knn = knn.fit(x_res, y_res)
    
    y_pred = knn.predict(X_test)
    r = recall_score(y_pred, y_test)
    recall_scores.append((k, round(recall_score(y_test, y_pred), 4)))
    f1_scores.append((k, round(f1_score(y_test, y_pred), 4)))
    
r_results = pd.DataFrame(recall_scores, columns=['K', 'Recall'])
f1_results = pd.DataFrame(f1_scores, columns=['K', 'F1'])

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')

plt.figure(dpi=300)
ax = r_results.set_index('K').plot(figsize=(12, 8), linewidth=6)
ax.set(xlabel='K', ylabel='Recall')
ax.set_xticks(range(1, max_k, 2));
plt.title('KNN recall')
plt.legend(labels='R')

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')

plt.figure(dpi=300)
ax = f1_results.set_index('K').plot(figsize=(12, 8), linewidth=6)
ax.set(xlabel='K', ylabel='f1_score')
ax.set_xticks(range(1, max_k, 2));
plt.title('KNN F1')
plt.legend(labels='f1_score')

From both the recall and F1 score we can see that k=13 can be considered as an optimal value

In [ ]:
_, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='viridis', annot_kws={"size": 40, "weight": "bold"})  
labels = ['True','False']
ax.set_xticklabels(labels, fontsize=25);
ax.set_yticklabels(labels, fontsize=25);
ax.set_ylabel('Prediction', fontsize=30);
ax.set_xlabel('Ground Truth', fontsize=30)
### END SOLUTION

In [ ]:
knn = KNeighborsClassifier(n_neighbors=13, weights='distance')
knn = knn.fit(x_res, y_res)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

KNN focused more heavily on identifying the False negatives

# SVC

For our next two models let's try and optimize for f1 score and see the results

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y,random_state=42)

In [ ]:
sm = SMOTE(random_state=42)
x_res,y_res=sm.fit_resample(X_train,y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
estimator = Pipeline([("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(max_iter=100000))])

params = {
    'linear_svc__C': np.linspace(0.001,100)
}
grid = GridSearchCV(estimator, params, cv=5,verbose=True,n_jobs=-1,scoring='f1')

In [ ]:
grid.fit(x_res,y_res)

In [ ]:
grid.best_params_

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
_, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='viridis', annot_kws={"size": 40, "weight": "bold"})  
labels = ['True','False']
ax.set_xticklabels(labels, fontsize=25);
ax.set_yticklabels(labels, fontsize=25);
ax.set_ylabel('Prediction', fontsize=30);
ax.set_xlabel('Ground Truth', fontsize=30)
### END SOLUTION

In [ ]:
print(classification_report(y_test, y_pred))

Similar results to that of Logistic Regression 

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,stratify=y,random_state=42)

In [ ]:
sm = SMOTE(random_state=42)
x_res,y_res=sm.fit_resample(X_train,y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
estimator = RandomForestClassifier(n_estimators=300)

params = { 'max_depth' : range(5,20),
          'min_samples_split' : np.arange(2,10)
}
grid = GridSearchCV(estimator, params, cv=5,verbose=True,n_jobs=-1,scoring='f1')

In [ ]:
grid.fit(x_res,y_res)

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
_, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='viridis', annot_kws={"size": 40, "weight": "bold"})  
labels = ['True','False']
ax.set_xticklabels(labels, fontsize=25);
ax.set_yticklabels(labels, fontsize=25);
ax.set_ylabel('Prediction', fontsize=30);
ax.set_xlabel('Ground Truth', fontsize=30)
### END SOLUTION

In [ ]:
grid.best_params_

In [ ]:
print(classification_report(y_test, y_pred))

Again similar results compared to the other models, with very little to no improvement in our scores

# Combining models 
Lets try and combine all our models and see how well it works then

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
lr = LogisticRegression(solver='liblinear',penalty='l1',C=2.0417959183673466)
knn = KNeighborsClassifier(n_neighbors=13, weights='distance')
svc = LinearSVC(penalty='l2',C=0.001)
rf = RandomForestClassifier(max_depth=8, min_samples_split=2,n_estimators=300)

In [ ]:
eclf = VotingClassifier(estimators=[('clf1',lr),('clf2',knn),('clf3',svc),('clf4',rf)])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42,stratify=y)

In [ ]:
sm = SMOTE(random_state=42)
x_res,y_res=sm.fit_resample(X_train,y_train)

In [ ]:
eclf.fit(x_res,y_res)

In [ ]:
y_pred = eclf.predict(X_test)

In [ ]:
_, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='viridis', annot_kws={"size": 40, "weight": "bold"})  
labels = ['False', 'True']
ax.set_xticklabels(labels, fontsize=25);
ax.set_yticklabels(labels[::-1], fontsize=25);
ax.set_ylabel('Prediction', fontsize=30);
ax.set_xlabel('Ground Truth', fontsize=30)
### END SOLUTION

In [ ]:
print(classification_report(y_test, y_pred))

Still the same results , which is not surprising considering that all the models had similar predictions

# Conclusion

All the models had similar performance with recall around 0.75, which is not bad considering the size of the dataset and the imbalance.

However perhaps employing unsupervised methods such as clustering could reveal more information and help us improve our scores.
More powerful ensemble methods such as XGboost could also yield better result and I would like to try that perhaps some other day.

Also other sampling methods might have yielded better results.

In the end I would like your feedback on how i could improve the performance, thank you.